# Cubeviz Demonstration Notebook

This notebook demonstrates the Cubeviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Cubeviz, can be found here: https://jdaviz.readthedocs.io/en/latest/cubeviz/

Import modules needed for this notebook.

In [ ]:
import warnings

from astropy.utils.data import download_file
from photutils import CircularAperture
from regions import PixCoord, CirclePixelRegion

from jdaviz import Cubeviz

We create a Cubeviz instance and show it.

In [ ]:
cubeviz = Cubeviz()
cubeviz.show()

This file is originally from https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz but has been modified to correct some inconsistencies in the way units are parsed. We ignore some warnings that appear during parsing.

In [ ]:
fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(fn)

The upper left viewer containing the FLUX cube is the default viewer that is accessible as follows.

In [ ]:
viewer = cubeviz.default_viewer

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on an image viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = cubeviz.get_interactive_regions()

In [ ]:
regions

It is also possible to programmatically pass a `regions` shape or a `photutils` aperture shape.

*NOTE: Sky regions are not yet supported for data cubes.*

In [ ]:
data = cubeviz.app.get_data_from_viewer('flux-viewer', 'contents[FLUX]')

In [ ]:
# photutils aperture
my_aper = CircularAperture((15, 10), r=5)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=35, y=40), radius=20)

my_regions = [my_aper, my_reg]
cubeviz.load_regions(my_regions)

To get the spectra:

In [ ]:
spectra_dict = cubeviz.specviz.get_spectra()
spectra_dict

This contains the masked spectrum extracted from "Subset 1" region.

In [ ]:
subset1_spec1d = spectra_dict['Subset 1']
subset1_spec1d